In [1]:
import polars as pl
from janitor import make_clean_names

In [2]:
df = pl.DataFrame(
    {
        "Aloha": range(3),
        "Bell Chart": range(3),
        "Animals@#$%^": range(3)
    }
)
df

Aloha,Bell Chart,Animals@#$%^
i64,i64,i64
0,0,0
1,1,1
2,2,2


In [ ]:
raw.with_columns(
    pl.col("raw").pipe(
        make_clean_names, object_type="polars", strip_accents=True
    )
)